In [1]:
import os
import glob
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plot

import sklearn.metrics as metrics

import tensorflow as tf
import keras
from keras import layers, regularizers

import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [ ]:
print('/n Loading Training Data...')
train_data = tf.keras.utils.image_dataset_from_directory(
    "", shuffle=True
)